# CRS Check

Goal of this notebook is to check the CRS of the datasets.

In [1]:
import pandas
import rasterio
import geopandas as gpd
import numpy
import os

#### File Paths

In [2]:
# Local Hard Drive

DATA_RAW = '/Users/cascade/Github/wastewater_ohi/data/raw/'

### Population rasters

In [3]:
# GHS-Pop

pop_fn = 'GHS_POP_E2015_GLOBE_R2019A_54009_1K_V1_0/GHS_POP_E2015_GLOBE_R2019A_54009_1K_V1_0.tif'
pop =  rasterio.open(DATA_RAW+pop_fn)

In [4]:
pop.meta

{'driver': 'GTiff',
 'dtype': 'float64',
 'nodata': -200.0,
 'width': 36082,
 'height': 18000,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.017453292519943295]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]'),
 'transform': Affine(1000.0, 0.0, -18041000.0,
        0.0, -1000.0, 9000000.0)}

In [5]:
# GHS-SMOD

smod_fn = 'GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V1_0/GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V1_0.tif'
smod = rasterio.open(DATA_RAW+smod_fn)

In [6]:
smod.meta

{'driver': 'GTiff',
 'dtype': 'int16',
 'nodata': -200.0,
 'width': 36082,
 'height': 18000,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.017453292519943295]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]'),
 'transform': Affine(1000.0, 0.0, -18041000.0,
        0.0, -1000.0, 9000000.0)}

### Basins

In [7]:
# file list

basins_in = os.path.join(DATA_RAW, "basins_laea/")
shps = [os.path.join(basins_in, fn) for fn in os.listdir(basins_in) if fn.endswith(".shp")]
basins_out = '/Users/cascade/Github/wastewater_ohi/data/interim/basins_crs/'

In [8]:
# Check current CRS

for shp in shps:
    fn = gpd.read_file(shp)
    print(shp)
    print(fn.crs)
    print('\n')

/Users/cascade/Github/wastewater_ohi/data/raw/basins_laea/au_bas.shp
{'proj': 'laea', 'lat_0': -15, 'lon_0': 135, 'x_0': 0, 'y_0': 0, 'datum': 'WGS84', 'units': 'm', 'no_defs': True}


/Users/cascade/Github/wastewater_ohi/data/raw/basins_laea/sa_bas.shp
{'proj': 'laea', 'lat_0': -15, 'lon_0': -60, 'x_0': 0, 'y_0': 0, 'datum': 'WGS84', 'units': 'm', 'no_defs': True}


/Users/cascade/Github/wastewater_ohi/data/raw/basins_laea/af_bas.shp
{'proj': 'laea', 'lat_0': 5, 'lon_0': 20, 'x_0': 0, 'y_0': 0, 'datum': 'WGS84', 'units': 'm', 'no_defs': True}


/Users/cascade/Github/wastewater_ohi/data/raw/basins_laea/as_bas.shp
{'proj': 'laea', 'lat_0': 45, 'lon_0': 100, 'x_0': 0, 'y_0': 0, 'datum': 'WGS84', 'units': 'm', 'no_defs': True}


/Users/cascade/Github/wastewater_ohi/data/raw/basins_laea/eu_bas.shp
{'proj': 'laea', 'lat_0': 55, 'lon_0': 20, 'x_0': 0, 'y_0': 0, 'datum': 'WGS84', 'units': 'm', 'no_defs': True}


/Users/cascade/Github/wastewater_ohi/data/raw/basins_laea/pa_bas.shp
{'proj': 'la

In [10]:
from fiona.crs import from_epsg
from_epsg(54009)

{'init': 'epsg:54009', 'no_defs': True}

In [18]:
# Write basins out into a new crs

#crs = pop.crs # <<<--- doesn't work

# from Africa ERL paper
#crs = {'proj': 'moll', 'lon_0': 0, 'x_0': 0, 'y_0': 0, 'ellps': 'WGS84', 'units': 'm', 'no_defs': True}



# Directly from fiona package based on the GHS website CRS <<<--- Not working
# from fiona.crs import from_epsg
# crs = from_epsg(54009)
# crs 

# Trying from here: https://epsg.io/54009
#crs = {'proj': 'moll', 'lon_0' :0, 'x_0': 0, 'y_0' :0, 'datum': 'WGS84', 'units': 'm', 'no_defs' : True}

In [21]:
# Trying again to get the crs from pop, validates the above code

pop.crs.to_proj4()

'+proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs=True'

In [22]:
for shp in shps:
    
    # Get file names and make new ones
    fn = shp.split(basins_in)[1].split('.')[0]
    fn_out = fn+'_4326.shp'

    basin = gpd.read_file(shp).to_crs(epsg = 4326)
    
    basin.to_file(basins_out+fn_out)